# 1. Perkenalan Dataset

Dataset ini memberikan gambaran menyeluruh tentang berbagai faktor yang memengaruhi kinerja siswa dalam ujian. Termasuk informasi tentang kebiasaan belajar, kehadiran, keterlibatan orang tua, dan aspek lain yang memengaruhi keberhasilan akademis.

## Deskripsi Kolom

|Atribut| Deskripsi |
|--|--|
| Hours_Studied | Jumlah jam yang dihabiskan untuk belajar per minggu. |
| Attendance | Persentase kelas yang dihadiri. |
| Parental_Involvement | Tingkat keterlibatan orang tua dalam pendidikan siswa (Low, Medium, High). |
| Access_to_Resources | Ketersediaan sumber daya pendidikan (Low, Medium, High). |
| Extracurricular_Activities | Partisipasi dalam kegiatan ekstrakurikuler (Ya, Tidak). |
| Sleep_Hours | Rata-rata jumlah jam tidur per malam. |
| Previous_Scores | Nilai ujian sebelumnya |
| Motivation_Level | Tingkat motivasi siswa (Low, Medium, High). |
| Internet_Access | Ketersediaan akses internet (Yes, No). |
| Tutoring_Sessions | Jumlah sesi bimbingan belajar yang dihadiri per bulan. |
| Family_Income | Tingkat pendapatan keluarga (Low, Medium, High). |
| Teacher_Quality | Kualitas Guru (Low, Medium, High) |
| School_Type | Jenis sekolah yang dihadiri (Public, Private) |
| Peer_Influence | Pengaruh teman sebaya terhadap prestasi akademik (Positive, Neutral, Negative) |
| Physical_Activity | Rata-rata jumlah jam aktivitas fisik per minggu. |
| Learning_Disabilities | Adanya kesulitan belajar (Yes, No). |
| Parental_Education_Level | Tingkat pendidikan tertinggi orang tua (High School, College, Postgraduate). |
| Distance_from_Home | Jarak dari rumah ke sekolah (Near, Moderate, Far). |
| Gender | Jenis kelamin siswa (Male, Female). |
| Exam_Score | Nilai ujian akhir. |


# 2. Import Library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from yellowbrick.cluster import KElbowVisualizer

# 3. Memuat Dataset

In [ ]:
ds = pd.read_csv('Dataset_inisiasi.csv')
ds.head()

# 4. Exploratory Data Analysis (EDA)

In [ ]:
# membuat salinan dataset
ds_cleaned = ds.copy()

## 1. Memahami Struktur Data

In [ ]:
# Menampilkan informasi tentang dataset, termasuk jumlah baris, kolom, tipe data, dan jumlah nilai non-null
ds_cleaned.info() 
ds_cleaned.shape

Dataset ini memiliki **6607 baris** data, dan **20 kolom**

In [ ]:
numerical_cols = ds_cleaned.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = ds_cleaned.select_dtypes(include=['object']).columns.tolist()

print(f'Jumlah Fitur: {len(categorical_cols) + len(numerical_cols)}')
print(f'Jumlah fitur numerik: {len(numerical_cols)}')
print(f'Jumlah fitur kategorikal: {len(categorical_cols)}\n')
print(f'Daftar fitur kategorikal:\n{categorical_cols}\n')
print(f'Daftar fitur numerikal:\n{numerical_cols}')

## 2. Menangani Data yang Hilang

In [ ]:
missing_values = ds_cleaned.isnull().sum()
percentage_missing_values = (missing_values / ds_cleaned.shape[0]) * 100

print(f'Total nilai yang kosong: {missing_values.sum()}')
print(f'Persentase nilai yang kosong: {percentage_missing_values.sum():.2f}%\n')

missing_values_ds = pd.DataFrame(percentage_missing_values[missing_values>0], columns=['percentage_missing_values'])
missing_values_ds['number_of_missing_values'] = missing_values[missing_values>0]

missing_values_ds

In [ ]:
sns.heatmap(ds.isnull(), cbar=False, cmap='viridis')

In [ ]:
# Drop the null values
ds_cleaned.dropna(axis=0, inplace=True)

missing_values = ds_cleaned.isnull().sum()
percentage_missing_values = (missing_values / ds_cleaned.shape[0]) * 100

print(f'Total nilai yang kosong: {missing_values.sum()}')
print(f'Persentase nilai yang kosong: {percentage_missing_values.sum():.2f}%\n')

missing_values_ds = pd.DataFrame(percentage_missing_values[missing_values>0], columns=['percentage_missing_values'])
missing_values_ds['number_of_missing_values'] = missing_values[missing_values>0]

missing_values_ds

Diketahui data yang hilang sebanyak 3.56% saja. Karena data yang hilang adalah kecil, jadi saya akan menangani data yang hilang dengan teknik drop pada baris yang memiliki data yang hilang.

## 3. Analisis Distribusi, Visualisasi, dan Korelasi

In [ ]:
# Menampilkan statistik deskriptif dari dataset
ds_cleaned.describe(include="all")

In [ ]:
# Menampilkan statistik deskriptif dari dataset yang hanya berisi fitur numerik
stat_desc_num = ds_cleaned[numerical_cols].describe().T
stat_desc_num

Berdasarkan output statistik deskriptif dari fitur numerik pada dataset, berikut adalah beberapa analisis yang dapat dilakukan:

1. **Hours_Studied (Jam Belajar)**
    - Rata-rata jam belajar siswa per minggu adalah sekitar 19.98 jam.
    - Jam belajar minimum adalah 1 jam dan maksimum adalah 44 jam.
    - Sebagian besar siswa belajar antara 16 hingga 24 jam per minggu (kuartil pertama hingga ketiga).

2. **Attendance (Kehadiran)**
    - Rata-rata persentase kehadiran siswa adalah sekitar 80.02%.
    - Persentase kehadiran minimum adalah 60% dan maksimum adalah 100%.
    - Sebagian besar siswa memiliki persentase kehadiran antara 70% hingga 90%.

3. **Sleep_Hours (Jam Tidur)**
    - Rata-rata jam tidur siswa per malam adalah sekitar 7.03 jam.
    - Jam tidur minimum adalah 4 jam dan maksimum adalah 10 jam.
    - Sebagian besar siswa tidur antara 6 hingga 8 jam per malam.

4. **Previous_Scores (Nilai Sebelumnya)**
    - Rata-rata nilai ujian sebelumnya adalah sekitar 75.07.
    - Nilai ujian sebelumnya minimum adalah 50 dan maksimum adalah 100.
    - Sebagian besar siswa memiliki nilai ujian sebelumnya antara 63 hingga 88.

5. **Tutoring_Sessions (Sesi Bimbingan)**
    - Rata-rata jumlah sesi bimbingan belajar yang dihadiri siswa per bulan adalah sekitar 1.50 sesi.
    - Jumlah sesi bimbingan minimum adalah 0 dan maksimum adalah 8 sesi.
    - Sebagian besar siswa menghadiri antara 1 hingga 2 sesi bimbingan per bulan.

6. **Physical_Activity (Aktivitas Fisik)**
    - Rata-rata jumlah jam aktivitas fisik siswa per minggu adalah sekitar 2.97 jam.
    - Jumlah jam aktivitas fisik minimum adalah 0 dan maksimum adalah 6 jam.
    - Sebagian besar siswa melakukan aktivitas fisik antara 2 hingga 4 jam per minggu.

7. **Exam_Score (Nilai Ujian)**
    - Rata-rata nilai ujian akhir siswa adalah sekitar 67.25.
    - Nilai ujian minimum adalah 55 dan maksimum adalah 101. Nilai maksimum akan diganti dengan 100, karena mungkin terjadi human error saat menginput nilai siswa.
    - Sebagian besar siswa memiliki nilai ujian antara 65 hingga 69.

Dari analisis ini, kita dapat melihat distribusi dan rentang nilai dari setiap fitur numerik dalam dataset. Informasi ini dapat membantu dalam memahami karakteristik data dan mengidentifikasi potensi anomali atau outlier yang mungkin memerlukan perhatian lebih lanjut.

In [ ]:
# mengganti nilai 101 menjadi 100 pada Exam_Score
ds_cleaned['Exam_Score'] = ds_cleaned['Exam_Score'].replace(101, 100)

# cek kembali data tipe numerikal
ds_cleaned[numerical_cols].describe().T

In [ ]:
# Menampilkan histogram dan plotbox distribusi dari fitur numerik
def plt_histogram_boxplot(ds_cleaned, columns=None, bins=30):
    
    if columns is None:
        columns = ds_cleaned.select_dtypes(include=['number']).columns
    
    num_cols = len(columns)
    fig, axes = plt.subplots(nrows=num_cols, ncols=2, figsize=(12, 5 * num_cols))
    for i, col in enumerate(columns):
        
        sns.histplot(ds_cleaned[col], bins=bins, kde=True, ax=axes[i, 0], color="blue")
        axes[i, 0].set_title(f'Histogram of {col}')
        axes[i, 0].set_xlabel(col)
        axes[i, 0].set_ylabel('Frequency')
        
        sns.boxplot(x=ds_cleaned[col], ax=axes[i, 1], color="red")
        axes[i, 1].set_title(f'Boxplot of {col}')
        axes[i, 1].set_xlabel(col)

    plt.tight_layout()
    plt.show()

histo_box = plt_histogram_boxplot(ds_cleaned)
histo_box

Berdasarkan hasil gambar histogram dan boxplot dari fitur-fitur numerik, berikut adalah analisis rentang nilai, kecenderungan pusat, potensi outliers, serta potensi skewness yang ada:

1. **Hours_Studied (Jam Belajar)**
    - **Rentang Nilai**: 1 hingga 44 jam per minggu.
    - **Kecenderungan Pusat**: Rata-rata sekitar 20 jam per minggu.
    - **Potensi Outliers**: Ada beberapa outliers di atas 40 jam per minggu.
    - **Potensi Skewness**: Distribusi cenderung sedikit skewed ke kanan (positif skewness).

2. **Attendance (Kehadiran)**
    - **Rentang Nilai**: 60% hingga 100%.
    - **Kecenderungan Pusat**: Rata-rata sekitar 80%.
    - **Potensi Outliers**: Tidak ada outliers yang signifikan.
    - **Potensi Skewness**: Distribusi cenderung normal.

3. **Sleep_Hours (Jam Tidur)**
    - **Rentang Nilai**: 4 hingga 10 jam per malam.
    - **Kecenderungan Pusat**: Rata-rata sekitar 7 jam per malam.
    - **Potensi Outliers**: Tidak ada outliers yang signifikan.
    - **Potensi Skewness**: Distribusi cenderung normal.

4. **Previous_Scores (Nilai Sebelumnya)**
    - **Rentang Nilai**: 50 hingga 100.
    - **Kecenderungan Pusat**: Rata-rata sekitar 75.
    - **Potensi Outliers**: Tidak ada outliers yang signifikan.
    - **Potensi Skewness**: Distribusi cenderung normal.

5. **Tutoring_Sessions (Sesi Bimbingan)**
    - **Rentang Nilai**: 0 hingga 8 sesi per bulan.
    - **Kecenderungan Pusat**: Rata-rata sekitar 1.5 sesi per bulan.
    - **Potensi Outliers**: Ada beberapa outliers di atas 6 sesi per bulan.
    - **Potensi Skewness**: Distribusi cenderung skewed ke kanan (positif skewness).

6. **Physical_Activity (Aktivitas Fisik)**
    - **Rentang Nilai**: 0 hingga 6 jam per minggu.
    - **Kecenderungan Pusat**: Rata-rata sekitar 3 jam per minggu.
    - **Potensi Outliers**: Tidak ada outliers yang signifikan.
    - **Potensi Skewness**: Distribusi cenderung normal.

7. **Exam_Score (Nilai Ujian)**
    - **Rentang Nilai**: 55 hingga 100.
    - **Kecenderungan Pusat**: Rata-rata sekitar 67.
    - **Potensi Outliers**: Ada beberapa outliers di bawah 60 dan di atas 90.
    - **Potensi Skewness**: Distribusi cenderung sedikit skewed ke kiri (negatif skewness).

Secara keseluruhan, sebagian besar fitur numerik memiliki distribusi yang cukup normal dengan beberapa outliers yang perlu diperhatikan, terutama pada fitur `Hours_Studied`, `Tutoring_Sessions`, dan `Exam_Score`. Outliers ini dapat mempengaruhi analisis lebih lanjut dan mungkin memerlukan penanganan khusus. Potensi skewness juga perlu diperhatikan karena dapat mempengaruhi hasil analisis statistik dan model prediktif.

In [ ]:
cat_desc = ds_cleaned[categorical_cols].describe().T
cat_desc['freq_percentage(%)'] = cat_desc['freq'] / ds_cleaned.shape[0] * 100
cat_desc

In [ ]:
# Visualisasi distribusi untuk variabel kategorikal
plt_cat = plt.figure(figsize=(15, 20))
for i, column in enumerate(categorical_cols, 1):
    plt.subplot(5, 3, i)
    sns.countplot(y=ds_cleaned[column], palette='viridis')
    plt.title(f'Distribusi {column}')
    plt.xlabel('Count')
    plt.ylabel(column)
plt.tight_layout()

plt_cat.show()

Berdasarkan hasil visualisasi distribusi untuk variabel kategorikal, berikut adalah analisisnya:

1. **Parental_Involvement (Keterlibatan Orang Tua)**
    - Distribusi keterlibatan orang tua menunjukkan bahwa sebagian besar siswa memiliki tingkat keterlibatan orang tua yang sedang (Medium).

2. **Access_to_Resources (Akses ke Sumber Daya)**
    - Sebagian besar siswa memiliki akses ke sumber daya pendidikan yang sedang (Medium).

3. **Extracurricular_Activities (Kegiatan Ekstrakurikuler)**
    - Sebagian besar siswa berpartisipasi dalam kegiatan ekstrakurikuler (Yes).
    - Jumlah siswa yang tidak berpartisipasi dalam kegiatan ekstrakurikuler (No) lebih sedikit.

4. **Motivation_Level (Tingkat Motivasi)**
    - Sebagian besar siswa memiliki tingkat motivasi yang sedang (Medium).

5. **Internet_Access (Akses Internet)**
    - Sebagian besar siswa memiliki akses internet (Yes).
    - Jumlah siswa yang tidak memiliki akses internet (No) lebih sedikit.

6. **Family_Income (Pendapatan Keluarga)**
    - Sebagian besar siswa berasal dari keluarga dengan pendapatan sedang (Medium).

7. **Teacher_Quality (Kualitas Guru)**
    - Sebagian besar siswa menilai kualitas guru mereka sebagai sedang (Medium).

8. **School_Type (Jenis Sekolah)**
    - Sebagian besar siswa bersekolah di sekolah umum (Public).
    - Jumlah siswa yang bersekolah di sekolah privat (Private) lebih sedikit.

9. **Peer_Influence (Pengaruh Teman Sebaya)**
    - Sebagian besar siswa memiliki pengaruh teman sebaya yang positif (Positive).
    - Pengaruh teman sebaya yang netral (Neutral) dan negatif (Negative) memiliki jumlah yang hampir sama.

10. **Learning_Disabilities (Kesulitan Belajar)**
    - Sebagian besar siswa tidak memiliki kesulitan belajar (No).
    - Jumlah siswa yang memiliki kesulitan belajar (Yes) lebih sedikit.

11. **Parental_Education_Level (Tingkat Pendidikan Orang Tua)**
    - Sebagian besar siswa memiliki orang tua dengan tingkat pendidikan sekolah menengah (High School).

12. **Distance_from_Home (Jarak dari Rumah)**
    - Sebagian besar siswa tinggal dekat dengan sekolah (Near).

13. **Gender (Jenis Kelamin)**
    - Distribusi jenis kelamin siswa menunjukkan jumlah yang hampir seimbang antara laki-laki (Male) dan perempuan (Female).

Secara keseluruhan, distribusi variabel kategorikal menunjukkan bahwa sebagian besar siswa memiliki karakteristik yang sedang (Medium) dalam berbagai aspek seperti keterlibatan orang tua, akses ke sumber daya, motivasi, pendapatan keluarga, kualitas guru, dan pengaruh teman sebaya. Selain itu, sebagian besar siswa memiliki akses internet, berpartisipasi dalam kegiatan ekstrakurikuler, tidak memiliki kesulitan belajar, dan tinggal dekat dengan sekolah.

In [ ]:
# Heatmap korelasi untuk fitur numerik
plt_corr_matrix = plt.figure(figsize=(12, 10))
correlation = ds_cleaned[numerical_cols].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Heatmap Korelasi')
plt.show()

Berdasarkan hasil visualisasi matriks korelasi dari fitur numerik, berikut adalah beberapa analisis yang dapat dilakukan:

1. **Hours_Studied (Jam Belajar)**
    - Korelasi positif yang cukup kuat dengan `Exam_Score` (0.45). Ini menunjukkan bahwa semakin banyak jam belajar siswa, semakin tinggi nilai ujian mereka.

2. **Attendance (Kehadiran)**
    - Korelasi positif yang cukup kuat dengan `Exam_Score` (0.58). Ini menunjukkan bahwa semakin tinggi persentase kehadiran siswa, semakin tinggi nilai ujian mereka.

3. **Sleep_Hours (Jam Tidur)**
    - Korelasi yang sangat lemah dengan semua fitur lainnya, termasuk `Exam_Score` (-0.02). Ini menunjukkan bahwa jumlah jam tidur siswa tidak memiliki pengaruh signifikan terhadap nilai ujian mereka.

4. **Previous_Scores (Nilai Sebelumnya)**
    - Korelasi positif yang lemah dengan `Exam_Score` (0.18). Ini menunjukkan bahwa nilai ujian sebelumnya memiliki sedikit pengaruh terhadap nilai ujian akhir siswa.

5. **Tutoring_Sessions (Sesi Bimbingan)**
    - Korelasi positif yang lemah dengan `Exam_Score` (0.16). Ini menunjukkan bahwa jumlah sesi bimbingan belajar yang dihadiri siswa memiliki sedikit pengaruh terhadap nilai ujian mereka.

6. **Physical_Activity (Aktivitas Fisik)**
    - Korelasi yang sangat lemah dengan semua fitur lainnya, termasuk `Exam_Score` (0.03). Ini menunjukkan bahwa jumlah jam aktivitas fisik siswa tidak memiliki pengaruh signifikan terhadap nilai ujian mereka.

7. **Exam_Score (Nilai Ujian)**
    - Korelasi positif yang cukup kuat dengan `Hours_Studied` (0.45) dan `Attendance` (0.58). Ini menunjukkan bahwa jam belajar dan kehadiran adalah faktor penting yang mempengaruhi nilai ujian siswa.

Secara keseluruhan, dari matriks korelasi ini dapat disimpulkan bahwa `Hours_Studied` dan `Attendance` adalah dua fitur yang memiliki korelasi positif yang cukup kuat dengan `Exam_Score`. Fitur-fitur lainnya memiliki korelasi yang lemah atau sangat lemah dengan `Exam_Score`, menunjukkan bahwa mereka mungkin tidak memiliki pengaruh signifikan terhadap nilai ujian siswa.

## Insight EDA

Berikut adalah beberapa insight yang diperoleh dari tahap Exploratory Data Analysis (EDA) pada notebook ini:

1. **Distribusi Fitur Numerik**:
    - **Hours_Studied (Jam Belajar)**: Rata-rata siswa belajar sekitar 20 jam per minggu. Terdapat beberapa outliers yang belajar lebih dari 40 jam per minggu.
    - **Attendance (Kehadiran)**: Rata-rata kehadiran siswa adalah 80%. Sebagian besar siswa memiliki kehadiran antara 70% hingga 90%.
    - **Sleep_Hours (Jam Tidur)**: Rata-rata siswa tidur sekitar 7 jam per malam. Distribusi jam tidur cenderung normal tanpa outliers signifikan.
    - **Previous_Scores (Nilai Sebelumnya)**: Rata-rata nilai ujian sebelumnya adalah 75. Distribusi nilai cenderung normal tanpa outliers signifikan.
    - **Tutoring_Sessions (Sesi Bimbingan)**: Rata-rata siswa menghadiri sekitar 1.5 sesi bimbingan per bulan. Terdapat beberapa outliers yang menghadiri lebih dari 6 sesi per bulan.
    - **Physical_Activity (Aktivitas Fisik)**: Rata-rata siswa melakukan aktivitas fisik sekitar 3 jam per minggu. Distribusi aktivitas fisik cenderung normal tanpa outliers signifikan.
    - **Exam_Score (Nilai Ujian)**: Rata-rata nilai ujian akhir siswa adalah 67. Terdapat beberapa outliers di bawah 60 dan di atas 90.

2. **Distribusi Fitur Kategorikal**:
    - Sebagian besar siswa memiliki tingkat keterlibatan orang tua, akses ke sumber daya, motivasi, pendapatan keluarga, dan kualitas guru yang sedang (Medium).
    - Sebagian besar siswa memiliki akses internet, berpartisipasi dalam kegiatan ekstrakurikuler, tidak memiliki kesulitan belajar, dan tinggal dekat dengan sekolah.
    - Distribusi jenis kelamin siswa menunjukkan jumlah yang hampir seimbang antara laki-laki dan perempuan.

3. **Korelasi Antar Fitur**:
    - **Hours_Studied (Jam Belajar)** dan **Attendance (Kehadiran)** memiliki korelasi positif yang cukup kuat dengan **Exam_Score (Nilai Ujian)**. Ini menunjukkan bahwa semakin banyak jam belajar dan semakin tinggi kehadiran siswa, semakin tinggi nilai ujian mereka.
    - Fitur-fitur lainnya memiliki korelasi yang lemah atau sangat lemah dengan **Exam_Score**, menunjukkan bahwa mereka mungkin tidak memiliki pengaruh signifikan terhadap nilai ujian siswa.

4. **Penanganan Data yang Hilang**:
    - Data yang hilang hanya sebesar 3.56%, sehingga dilakukan teknik drop pada baris yang memiliki data yang hilang.

5. **Anomali Nilai**:
    - Terdapat anomali nilai maksimum pada **Exam_Score** yang mencapai 101. Nilai ini telah diganti menjadi 100 untuk mengatasi kemungkinan kesalahan input.

Insight-insight ini memberikan gambaran menyeluruh tentang karakteristik data dan hubungan antar fitur, yang dapat membantu dalam analisis lebih lanjut dan pembuatan model prediktif.

# 5. Data Preprocessing

## 1. Mengidentifikasi data duplikat

In [ ]:
total_duplicated = ds_cleaned.duplicated().sum()

if total_duplicated > 0:
    print(f'Total baris duplikat: {total_duplicated}')
else:
    print(f'Total baris duplikat rows: {total_duplicated}')
    print(f'Tidak ada baris duplikat')

## 2. Scaling

In [ ]:
ds_scaled = ds_cleaned.copy()
scaler = MinMaxScaler()

ds_scaled[numerical_cols] = scaler.fit_transform(ds_cleaned[numerical_cols])
ds_scaled[numerical_cols]

In [ ]:
# Menampilkan histogram dan plotbox distribusi dari fitur numerik
def plt_histogram_boxplot(ds_cleaned, columns=None, bins=30):
    
    if columns is None:
        columns = ds_cleaned.select_dtypes(include=['number']).columns
    
    num_cols = len(columns)
    fig, axes = plt.subplots(nrows=num_cols, ncols=2, figsize=(12, 5 * num_cols))
    for i, col in enumerate(columns):
        
        sns.histplot(ds_cleaned[col], bins=bins, kde=True, ax=axes[i, 0], color="blue")
        axes[i, 0].set_title(f'Histogram of {col}')
        axes[i, 0].set_xlabel(col)
        axes[i, 0].set_ylabel('Frequency')
        
        sns.boxplot(x=ds_cleaned[col], ax=axes[i, 1], color="red")
        axes[i, 1].set_title(f'Boxplot of {col}')
        axes[i, 1].set_xlabel(col)

    plt.tight_layout()
    plt.show()

plt_histogram_boxplot(ds_scaled)

In [ ]:
ds_scaled[numerical_cols].describe().T

Memang masih terdapat outliers pada data. Namun, outlier pada data ini bukanlah karena error, melainkan adalah sebuah informasi yang berguna, misalnya pada fitur Exam_Score nilai yang diatas 80 merupakan outlier. Jadi saya tidak akan membersihkan data outlier. 

## 3. Encoding Data Kategorikal

In [ ]:
ds_scaled[categorical_cols].info()

In [ ]:
ds_scaled[categorical_cols].describe().T

Saya mengidentifikasi 2 tipe data kategorikal pada scaled_ds
- Nominal :
    extracurricular_activities, internet_access, school_type, peer_influence, learning_disabilities, gender
- Ordinal :
    parental_involvement, access_to_resources, motivational_level, family_income, teacher_quality, parental_education_level, distance_from_home, hours_studied_binned, exam_score_binned, tutoring_sessions_binned

Saya akan melakukan OneHotEncode pada data bertipe nominal, dan OrdinalEncoder pada data bertipe ordinal

In [ ]:
ordinal_features = ['Parental_Involvement',
                    'Access_to_Resources',
                    'Motivation_Level',
                    'Family_Income',
                    'Teacher_Quality',
                    'Parental_Education_Level',
                    'Distance_from_Home']
nominal_features = ds_scaled[categorical_cols].columns.difference(ordinal_features).tolist()
print(f'Fitur Ordinal: {ordinal_features}\n')
print(f'Fitur Nominal: {nominal_features}')

In [ ]:
print(f'Fitur ordinal sebelum encoding: ')
ds_encoded = ds_scaled.copy()
ds_encoded[ordinal_features].head()

In [ ]:
print(f'Nilai unik pada data ordinal: ')
for features in ordinal_features:
    print(f'{features} : {ds_encoded[features].unique()}')

Encoding pada data ordinal harus dilakukan menetapkan urutan kategori. Setiap fitur ordinal mempunyai nilai unik dan urutan uniknya, yaitu sebagai berikut:
1. Tipe satu : Fitur ordinal dari parental_involvement ,access_to_resources, motivation_level, family_income, teacher_quality, hours_studied_binned, tutoring_sessions_binned dengan nilai dan urutan Low < Medium < High
2. Tipe dua : Fitur ordinal dari parental_education_level dengan nilai dan urutan School < College < Postgraduate
3. Tipe tiga : Fitur ordinal dari distance_from_home dengan nilai dan urutan Near < Moderate < Far
4. Tipe empat : exam_score_binned dengan nilai dan urutan D < C < B < A

In [ ]:
# masukkan tiap tipe nilai ordinal 
type_one = [col for col in ordinal_features if 'Medium' in ds_encoded[col].unique()]
type_two = [col for col in ordinal_features if 'College' in ds_encoded[col].unique()]
type_three = [col for col in ordinal_features if 'Moderate' in ds_encoded[col].unique()]

ordinal_type = [type_one, type_two, type_three]

# tentukan setiap kategori pada setiap tipe ordinal
categories = [
    ['Low', 'Medium', 'High'],
    ['High School', 'College', 'Postgraduate'],
    ['Near', 'Moderate', 'Far']
]

print(f'Tipe dalam fitur ordinal:')
for i, tipe_i in enumerate(ordinal_type):
    print(f'Tipe {i+1}: {tipe_i}\n Nilai: {categories[i]}\n')

In [ ]:
# Fitur ordinal sebelum encoding
print(f'Fitur ordinal sebelum encoding: ')
ds_cleaned[ordinal_features].head()

In [ ]:
# Encoding fitur ordinal
ordinal_encoders = {}
for i, class_i in enumerate(ordinal_type):
    encoder = OrdinalEncoder(categories=[categories[i]]*len(class_i), dtype=np.int64)
    ds_encoded[class_i] = encoder.fit_transform(ds_encoded[class_i])
    ordinal_encoders[f'type_{i+1}'] = encoder

# Menampilkan hasil encoding
print(f'Fitur ordinal setelah encoding: ')
ds_encoded[ordinal_features].head()

In [ ]:
# Melakukan one-hot encoding pada fitur nominal
for col in nominal_features:
    dummies = pd.get_dummies(ds_cleaned[col], dtype=int, prefix=col)
    ds_encoded = pd.concat([ds_encoded, dummies], axis=1)

ds_encoded.drop(nominal_features, axis=1, inplace=True)
ds_encoded.info()

In [ ]:
ds_encoded.head()

## Insight Data Preprocessing

Berikut adalah beberapa insight yang diperoleh dari tahap Data Preprocessing pada notebook ini:

1. **Identifikasi Data Duplikat**:
    - Tidak ditemukan baris duplikat dalam dataset. Hal ini menunjukkan bahwa data yang digunakan tidak memiliki duplikasi yang dapat mempengaruhi analisis lebih lanjut.

2. **Scaling (Normalisasi Data)**:
    - Fitur numerik dalam dataset dinormalisasi menggunakan MinMax. Hal ini dilakukan untuk memastikan bahwa semua fitur memiliki skala yang sama, sehingga tidak ada fitur yang mendominasi yang lain dalam analisis lebih lanjut.
    - Setelah normalisasi, fitur-fitur numerik memiliki distribusi yang lebih seimbang dan siap untuk digunakan dalam algoritma machine learning.

3. **Encoding Data Kategorikal**:
    - Fitur kategorikal dalam dataset diencode menggunakan dua metode:
        - OneHotEncoder untuk fitur nominal, yang menghasilkan kolom biner untuk setiap kategori.
        - OrdinalEncoder untuk fitur ordinal, yang mengubah kategori menjadi nilai numerik berdasarkan urutan yang telah ditentukan.
    - Encoding ini memungkinkan algoritma machine learning untuk memahami dan memproses fitur kategorikal dengan lebih baik.

Secara keseluruhan, tahap Data Preprocessing ini memastikan bahwa dataset siap untuk digunakan dalam analisis lebih lanjut dan pembuatan model machine learning. Data yang telah dibersihkan, dinormalisasi, dan diencode akan memberikan hasil yang lebih akurat dan dapat diandalkan.

# 6. Pembangunan Model Clustering

## a. Pembangunan Model Clustering

Saya akan menggunakan algoritma K-means clustering.

In [ ]:
# copy dataframe hasil encoding
ds_clustered = ds_encoded.copy()
ds_clustered.info()

In [ ]:
categorical_based = ds_clustered.select_dtypes(include=['int64']).columns
numerical_based = ds_clustered.select_dtypes(include=['float64']).columns

In [ ]:
# KMeans Clustering
kmeans_base = KMeans()
kmeans_base.fit(ds_clustered)
kmeans_labels = kmeans_base.labels_

# Jumlah kluster dan noise pada KMeans
kmeans_clusters = len(set(kmeans_labels))
kmeans_noise = list(kmeans_labels).count(-1)
print(f'Jumlah kluster pada KMeans: {kmeans_clusters}')
print(f'Jumlah noise pada KMeans: {kmeans_noise}')

## b. Model Evaluation

Untuk menentukan jumlah cluster yang optimal dalam model clustering, saya menggunakan metode Elbow atau Silhouette Score.

Metode ini membantu untuk menemukan jumlah cluster yang memberikan pemisahan terbaik antar kelompok data, sehingga model yang dibangun dapat lebih efektif. Berikut adalah **rekomendasi** tahapannya.
1. Gunakan Silhouette Score dan Elbow Method untuk menentukan jumlah cluster optimal.
2. Hitung Silhouette Score sebagai ukuran kualitas cluster.

In [ ]:
# Evaluasi KMeans hasil dengan Silhouette Score
score = silhouette_score(ds_clustered, kmeans_labels)
print(f'Silhouette Score KMeans: {score:.4f}')

In [ ]:
# mencari nilai optimal untuk model KMeans dengan Elbow Method
kmeans_base_elbow = KMeans()
visualizer = KElbowVisualizer(kmeans_base_elbow, k=(1,40))
visualizer.fit(ds_clustered) 
visualizer.show() 

plt.show()

Visualisasi di atas memperlihatkan bahwa nilai k yang terbaik adalah 13

In [ ]:
# perbarui jumlah kluster Kmeans menggunakan hasil elbow method
kmeans_base_elbow = KMeans(n_clusters=13, random_state=42)
kmeans_base_elbow.fit(ds_clustered)
kmeans_labels = kmeans_base_elbow.labels_

# Evaluasi KMeans hasil dengan Silhouette Score
score = silhouette_score(ds_clustered, kmeans_labels)
print(f'Silhouette Score KMeans Setelah Elbow Method: {score:.4}')

# Jumlah kluster dan noise pada KMeans
kmeans_clusters = len(set(kmeans_labels))
kmeans_noise = list(kmeans_labels).count(-1)
print(f'Jumlah kluster pada KMeans: {kmeans_clusters}')
print(f'Jumlah noise pada KMeans: {kmeans_noise}')

Silhoutte score masih rendah. Saya akan melakukan feature selection berdasarkan features variancenya agar menghasilkan model prediksi yang baik.

## c. Feature Selection (Opsional)

In [ ]:
#cek features variance
features_variance = np.var(ds_clustered, axis=0).sort_values(ascending=False)
features_variance = pd.Series(features_variance, index=ds_clustered.columns).sort_values(ascending=False)

print(f'Feature variances: ')
features_variance

In [ ]:
# pilih features variance yang lebih dari 0.5
ds_selected = ds_clustered.loc[:, features_variance > 0.5]
ds_selected

In [ ]:
# lakukan optimasi jumlah kluster dengan elbow method pada features yang dipilih
kmeans_selected_features_elbow = KMeans()
visualizer = KElbowVisualizer(kmeans_selected_features_elbow, k=(1,40))
visualizer.fit(ds_selected) 
visualizer.show()  
plt.show()

In [ ]:
# perbarui jumlah kluster Kmeans dari fitur yang dipilih menggunakan feature variance
kmeans_selected_features_elbow = KMeans(n_clusters=8, random_state=42)
kmeans_selected_features_elbow.fit(ds_selected)

kmeans_labels_selected = kmeans_selected_features_elbow.labels_
score = silhouette_score(ds_selected, kmeans_labels_selected)
print(f'Silhouette Score KMeans Setelah Elbow Method: {score:.4}')

# Jumlah kluster dan noise 
kmeans_clusters = len(set(kmeans_labels_selected))
kmeans_noise = list(kmeans_labels_selected).count(-1)
print(f'Jumlah kluster pada KMeans: {kmeans_clusters}')
print(f'Jumlah noise pada KMeans: {kmeans_noise}')

In [ ]:
#melakukan feature selection dengan PCA 2 dimensi
pca = PCA(n_components=2, random_state=42)

pca.fit(ds_clustered)
pca.explained_variance_ratio_
sum(pca.explained_variance_ratio_)

In [ ]:
ds_reduced = pca.transform(ds_clustered)
ds_reduced = pd.DataFrame(ds_reduced, columns=['PCA1', 'PCA2'])
ds_reduced

In [ ]:
#optimasi jumlah kluster dengan elbow method pada data yang direduksi (PCA)
kmeans_reduced_PCA = KMeans()
visualizer = KElbowVisualizer(kmeans_reduced_PCA, k=(1,40))
visualizer.fit(ds_reduced)  # Fit the data to the visualizer
visualizer.show()  # Finalize the plot
plt.show()

In [ ]:
# perbarui jumlah kluster Kmeans menggunakan hasil elbow method (PCA)
kmeans_reduced_PCA = KMeans(n_clusters=8, random_state=42)
kmeans_reduced_PCA.fit(ds_reduced)

kmeans_labels_reduced = kmeans_reduced_PCA.labels_
score = silhouette_score(ds_reduced, kmeans_labels_reduced)
print(f'Silhouette Score KMeans PCA Setelah Elbow Method: {score:.4}')

# Jumlah kluster dan noise pada KMeans
kmeans_clusters = len(set(kmeans_labels_reduced))
kmeans_noise = list(kmeans_labels_reduced).count(-1)
print(f'Jumlah kluster pada KMeans: {kmeans_clusters}')
print(f'Jumlah noise pada KMeans: {kmeans_noise}')

## d. Visualisasi Hasil Clustering

In [ ]:
#persiapan plotting data hasil clustering dengan PCA KMeans
kmeans_reduced_PCA = KMeans(n_clusters=8, random_state=42)
y_kmeans_reduced = kmeans_reduced_PCA.fit_predict(ds_reduced)
kmeans_reduced_cluster = ds_reduced.copy()
kmeans_reduced_cluster['cluster'] = y_kmeans_reduced
kmeans_reduced_cluster_mean = kmeans_reduced_cluster.groupby('cluster').mean()

kmeans_reduced_cluster_mean

In [ ]:
#persiapan plotting data hasil clustering dengan KMeans dengan pemilihan fitur berdsarkan nilai variance fitur
y_kmeans_selected = kmeans_selected_features_elbow.fit_predict(ds_selected)
kmeans_selected_cluster = ds_selected.copy()
kmeans_selected_cluster['cluster'] = y_kmeans_selected
kmeans_selected_cluster_mean = kmeans_selected_cluster.groupby('cluster').mean()

kmeans_selected_cluster_mean

In [ ]:
# Scatter plot 2D hasil clustering
plt.figure(figsize=(10, 8))

# Scatter plot data points
plt.scatter(ds_selected['Family_Income'], ds_selected['Parental_Education_Level'], 
            c=kmeans_labels_selected, cmap='viridis', alpha=0.5, label='Data Points', s=60,)

# Scatter plot centroids
plt.scatter(kmeans_selected_cluster_mean['Family_Income'], kmeans_selected_cluster_mean['Parental_Education_Level'], 
            c='red', marker='X', s=20, label='Centroids', alpha=0.8)

# Labeling axes and title
plt.xlabel('Family_Income')
plt.ylabel('Parental_Education_Level')
plt.title('Scatter Plot Hasil Clustering KMeans Berdasarkan Feature Variance')
plt.show()

In [ ]:
# Scatter plot 2D hasil clustering
plt.figure(figsize=(10, 8))

# Scatter plot data points
plt.scatter(ds_reduced['PCA1'], ds_reduced['PCA2'], 
            c=kmeans_labels_reduced, cmap='viridis', alpha=0.5, label='Data Points', s=60,)

# Scatter plot centroids
plt.scatter(kmeans_reduced_cluster_mean['PCA1'], kmeans_reduced_cluster_mean['PCA2'], 
            c='red', marker='X', s=20, label='Centroids', alpha=0.8)

# Labeling axes and title
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.title('Scatter Plot Hasil Clustering KMeans Berdasarkan Feature Variance')
plt.show()

Model clustering dengan Kmeans hasil seleksi fitur berdasarkan variance dan reduksi fitur dengan teknik PCA sama sama mendapatkan 8 cluster. Model Kmeans hasil seleksi fitur berdasarkan variance memiliki nilai silhoutte yang lebih besar, yaitu 0.94 persen, sedangkan dengan reduksi fitur (PCA) meiliki nilai silhoutte 0.8 persen. Karena itu saya memilih hasil clustering model KMeans dengan seleksi fitur berdasarkan nilai variance di atas 0.5. Fitur tersebut adalah Family_Income dan Parental_Education_Level.

## e. Analisis dan Interpretasi Hasil ClusterIn

In [ ]:
kmeans_selected_cluster_mean['Family_Income'] = kmeans_selected_cluster_mean['Family_Income'].apply(
    lambda x: 'Low' if x == 0.0 else 'Medium' if x == 1.0 else 'High'
)

kmeans_selected_cluster_mean['Parental_Education_Level'] = kmeans_selected_cluster_mean['Parental_Education_Level'].apply(
    lambda x: 'High_School' if x == 0.0 else 'College' if x == 1.0 else 'Postgraduate'
)

kmeans_selected_cluster_mean

In [ ]:
#inverse encoded nominal features to original
for col in nominal_features:
    onehot_cols = [c for c in ds_clustered.columns if c.startswith(col + '_')]
    ds_clustered[col] = ds_clustered[onehot_cols].idxmax(axis=1).apply(lambda x: x.split('_')[-1])
    ds_clustered.drop(columns=onehot_cols, inplace=True)

In [ ]:
#inverse encoded ordinal features to original
for i, class_i in enumerate(ordinal_type):
    encoder = ordinal_encoders[f'type_{i+1}']
    ds_clustered[class_i] = encoder.inverse_transform(ds_clustered[class_i])

In [ ]:
#inverse scaled numerical features to original
inverse_scaled_ds = pd.DataFrame(
    scaler.inverse_transform(ds_clustered[numerical_cols]), 
    columns=numerical_cols
)

# Reattach categorical columns from ds_clustered
ds_cluster = inverse_scaled_ds.copy()
ds_cluster[categorical_cols] = ds_clustered[categorical_cols].values

# Display the result
ds_cluster.head()

In [ ]:
#join cluster label to original dataset
clusters=pd.concat([ds_cluster, pd.DataFrame({'cluster':kmeans_labels_selected})], axis=1)
clusters.head()

In [ ]:
# Visualisasi distribusi untuk seluruh fitur berdasarkan cluster
for c in clusters:
    grid= sns.FacetGrid(clusters, col='cluster')
    grid.map(plt.hist, c)

Karena fitur pada dataset ada banyak, maka saya hanya akan fokus terhadap beberapa fitur saja untuk dilakukan analisis karakteristik tiap cluster, yaitu Exam_Score, Hours_Studied, Attendance, Sleep_Hours, Family_Income, Parental_Education_Level, Peer_Influence.

In [ ]:
# Perform cluster analysis with numerical and categorical features
cluster_analysis = clusters.groupby('cluster').agg({
    'Exam_Score': ['mean', 'max', 'min'],
    'Hours_Studied': ['mean', 'max', 'min'],
    'Attendance': ['mean', 'max', 'min'],
    'Sleep_Hours': ['mean', 'max', 'min'],
    'Family_Income': lambda x: x.mode().iloc[0],  # Most common category
    'Parental_Education_Level': lambda x: x.mode().iloc[0],  # Most common category
    'Peer_Influence': lambda x: x.mode().iloc[0],  # Most common category
    'cluster': 'count'  # Frequency of each cluster
})

# Menambahkan "_" pada setiap spasi di nama kolom
cluster_analysis.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in cluster_analysis.columns]

# Round numerical columns
for col in ['Exam_Score_mean', 'Hours_Studied_mean', 'Attendance_mean', 'Sleep_Hours_mean']:  
    if col in cluster_analysis.columns:
        cluster_analysis[col] = cluster_analysis[col].round(3)

# Menyesuaikan nama kolom
cluster_analysis.rename(columns={
    'Family_Income_<lambda>': 'Family_Income',
    'Parental_Education_Level_<lambda>': 'Parental_Education_Level',
    'Peer_Influence_<lambda>': 'Peer_Influence_',
    'cluster_count': 'frequency'  # Rename cluster count
}, inplace=True)

# verifikasi nama kolom
print(cluster_analysis.columns)

# menampilkan hasil analisis kluster
cluster_analysis


**Hasil Interpretasi**
**Cluster 1**
- **Interpretasi:**  
  Siswa dalam cluster ini berasal dari keluarga berpenghasilan rendah tetapi memiliki motivasi belajar yang cukup baik. Kehadiran mereka cukup tinggi, dan pengaruh teman sebaya cenderung mendukung akademik mereka.  

---

**Cluster 2** 
- **Interpretasi:**  
  Siswa dalam cluster ini memiliki latar belakang keluarga yang lebih baik secara ekonomi dan pendidikan. Pengaruh teman sebaya yang positif mungkin membantu mereka mencapai nilai ujian yang lebih tinggi meskipun mereka belajar sedikit lebih sedikit dibanding cluster lain.  

---

**Cluster 3**
- **Interpretasi:**  
  Siswa dalam cluster ini memiliki tingkat belajar yang cukup tinggi, tetapi pengaruh teman sebaya mereka tidak terlalu mempengaruhi akademik mereka.  

---

**Cluster 4**
- **Interpretasi:**  
  Siswa dalam cluster ini berasal dari keluarga ekonomi rendah, tetapi orang tua memiliki tingkat pendidikan yang lebih tinggi. Namun, pengaruh teman sebaya yang netral mungkin menyebabkan mereka kurang terdorong untuk belajar lebih giat.  

---

**Cluster 5**
- **Interpretasi:**  
  Siswa dalam cluster ini mungkin memiliki akses ke fasilitas pendidikan yang lebih baik karena kondisi ekonomi keluarga yang lebih baik, meskipun pendidikan orang tua masih di tingkat SMA.  

---

**Cluster 6**
- **Interpretasi:**  
  Siswa dalam cluster ini berasal dari latar belakang ekonomi rendah, tetapi orang tua mereka memiliki pendidikan yang lebih tinggi, yang kemungkinan memberikan dorongan tambahan bagi mereka untuk berprestasi.  

---

**Cluster 7**
- **Interpretasi:**  
  Siswa dalam cluster ini berasal dari latar belakang ekonomi menengah dengan orang tua yang berpendidikan tinggi. Pengaruh teman sebaya yang positif mungkin membantu mereka tetap termotivasi dalam belajar meskipun jumlah jam belajar mereka tidak terlalu tinggi.  

---

**Cluster 8**
- **Interpretasi:**  
  Siswa dalam cluster ini memiliki latar belakang ekonomi dan pendidikan orang tua yang tinggi, yang mungkin memberi mereka akses ke sumber belajar yang lebih baik. Pengaruh teman sebaya yang positif semakin mendorong mereka untuk mencapai hasil akademik terbaik.  

### **Analisis Karakteristik Cluster dari Model KMeans**  

Berikut adalah analisis karakteristik untuk setiap cluster berdasarkan skor ujian, jumlah jam belajar, tingkat kehadiran, durasi tidur, serta faktor sosial dan ekonomi lainnya.  

#### **Cluster 1**  
- **Rata-rata Skor Ujian:** 66.50  
- **Rata-rata Jam Belajar:** 20.02  
- **Rata-rata Kehadiran:** 79.93%  
- **Rata-rata Jam Tidur:** 7.01  
- **Pendapatan Orang Tua:** Rendah  
- **Pendidikan Orang Tua:** SMA  
- **Pengaruh Teman:** Positif  
- **Analisis:**  
  Siswa dalam cluster ini memiliki tingkat kehadiran yang cukup tinggi dan jam belajar yang relatif lebih tinggi. Namun, meskipun memiliki pengaruh teman yang positif, mereka berasal dari keluarga berpenghasilan rendah dan pendidikan orang tua yang terbatas. Hal ini menunjukkan bahwa mereka tetap mampu meraih skor ujian yang cukup baik meskipun memiliki keterbatasan ekonomi.  

#### **Cluster 2**  
- **Rata-rata Skor Ujian:** 68.13  
- **Rata-rata Jam Belajar:** 19.96  
- **Rata-rata Kehadiran:** 79.92%  
- **Rata-rata Jam Tidur:** 7.02  
- **Pendapatan Orang Tua:** Menengah  
- **Pendidikan Orang Tua:** Pascasarjana  
- **Pengaruh Teman:** Positif  
- **Analisis:**  
  Cluster ini terdiri dari siswa dengan pendidikan orang tua yang tinggi dan pengaruh teman yang positif. Hal ini tampaknya membantu mereka mencapai skor ujian yang lebih tinggi dibandingkan beberapa cluster lainnya. Dengan rata-rata jam belajar yang stabil dan kehadiran yang cukup baik, mereka menunjukkan performa akademik yang solid.  

#### **Cluster 3**  
- **Rata-rata Skor Ujian:** 67.05  
- **Rata-rata Jam Belajar:** 20.20  
- **Rata-rata Kehadiran:** 79.64%  
- **Rata-rata Jam Tidur:** 7.06  
- **Pendapatan Orang Tua:** Menengah  
- **Pendidikan Orang Tua:** SMA  
- **Pengaruh Teman:** Netral  
- **Analisis:**  
  Siswa dalam cluster ini memiliki jam belajar yang lebih tinggi dibandingkan rata-rata, tetapi skor ujian mereka tidak jauh berbeda dari cluster lain. Pengaruh teman yang netral mungkin berkontribusi pada stabilitas hasil akademik mereka, sementara latar belakang ekonomi menengah juga memberikan dukungan yang cukup.  

#### **Cluster 4**  
- **Rata-rata Skor Ujian:** 66.86  
- **Rata-rata Jam Belajar:** 19.75  
- **Rata-rata Kehadiran:** 79.89%  
- **Rata-rata Jam Tidur:** 7.09  
- **Pendapatan Orang Tua:** Rendah  
- **Pendidikan Orang Tua:** Perguruan Tinggi  
- **Pengaruh Teman:** Netral  
- **Analisis:**  
  Meskipun berasal dari keluarga berpenghasilan rendah, siswa dalam cluster ini memiliki orang tua dengan pendidikan yang cukup tinggi. Namun, pengaruh teman yang netral mungkin menyebabkan kurangnya dorongan sosial, yang berkontribusi pada skor ujian yang sedikit lebih rendah dibandingkan beberapa cluster lainnya.  

#### **Cluster 5**  
- **Rata-rata Skor Ujian:** 67.42  
- **Rata-rata Jam Belajar:** 19.79  
- **Rata-rata Kehadiran:** 79.59%  
- **Rata-rata Jam Tidur:** 6.94  
- **Pendapatan Orang Tua:** Tinggi  
- **Pendidikan Orang Tua:** SMA  
- **Pengaruh Teman:** Netral  
- **Analisis:**  
  Cluster ini menunjukkan bahwa siswa dengan pendapatan orang tua yang tinggi tidak selalu memiliki skor akademik yang lebih tinggi. Dengan tingkat kehadiran dan jam belajar yang mirip dengan cluster lain, faktor pengaruh teman yang netral mungkin menjadi salah satu penyebab tidak adanya perbedaan signifikan dalam pencapaian akademik mereka.  

#### **Cluster 6**  
- **Rata-rata Skor Ujian:** 67.74  
- **Rata-rata Jam Belajar:** 19.97  
- **Rata-rata Kehadiran:** 81.46%  
- **Rata-rata Jam Tidur:** 7.09  
- **Pendapatan Orang Tua:** Rendah  
- **Pendidikan Orang Tua:** Pascasarjana  
- **Pengaruh Teman:** Positif  
- **Analisis:**  
  Siswa dalam cluster ini memiliki tingkat kehadiran yang lebih tinggi dibandingkan cluster lainnya, serta pengaruh teman yang positif. Pendidikan orang tua yang tinggi juga mungkin menjadi faktor pendukung utama bagi mereka untuk meraih skor ujian yang cukup baik, meskipun berasal dari keluarga berpenghasilan rendah.  

#### **Cluster 7**  
- **Rata-rata Skor Ujian:** 67.54  
- **Rata-rata Jam Belajar:** 19.89  
- **Rata-rata Kehadiran:** 80.35%  
- **Rata-rata Jam Tidur:** 7.02  
- **Pendapatan Orang Tua:** Menengah  
- **Pendidikan Orang Tua:** Perguruan Tinggi  
- **Pengaruh Teman:** Positif  
- **Analisis:**  
  Dengan tingkat kehadiran yang relatif tinggi dan pengaruh teman yang positif, siswa dalam cluster ini menunjukkan kestabilan dalam pencapaian akademik. Latar belakang ekonomi menengah dan pendidikan orang tua yang cukup baik turut mendukung kondisi akademik mereka.  

#### **Cluster 8**  
- **Rata-rata Skor Ujian:** 68.04  
- **Rata-rata Jam Belajar:** 20.10  
- **Rata-rata Kehadiran:** 80.15%  
- **Rata-rata Jam Tidur:** 7.05  
- **Pendapatan Orang Tua:** Tinggi  
- **Pendidikan Orang Tua:** Perguruan Tinggi  
- **Pengaruh Teman:** Positif  
- **Analisis:**  
  Cluster ini memiliki siswa dengan skor ujian yang relatif lebih tinggi, tingkat kehadiran yang baik, serta dukungan sosial dan ekonomi yang kuat. Pengaruh teman yang positif dan pendidikan orang tua yang lebih tinggi tampaknya menjadi faktor utama yang membantu mereka meraih performa akademik yang lebih baik.  

---

### **Kesimpulan**  
1. **Pendapatan orang tua tidak selalu menentukan skor akademik.** Cluster dengan pendapatan rendah tetapi memiliki pengaruh teman yang positif dan pendidikan orang tua yang tinggi menunjukkan hasil akademik yang baik.  
2. **Kehadiran dan pengaruh teman memiliki dampak signifikan terhadap performa akademik.** Cluster dengan pengaruh teman yang positif dan tingkat kehadiran tinggi cenderung memiliki skor ujian yang lebih baik.  
3. **Jam belajar yang lebih tinggi tidak selalu berarti skor ujian yang lebih baik.** Beberapa cluster dengan jam belajar yang mirip menunjukkan variasi skor ujian yang berbeda, menunjukkan bahwa kualitas belajar dan dukungan sosial juga berperan penting.  



## 7. Mengeksport Data

In [ ]:
#simpan dataframe hasil clustering ke dalam file csv
clusters.to_csv('Dataset_clustering.csv')